In [8]:
from util.scraper import scrap

from util.file_handler import saveAsFileWithGenerator, make_file_path

from bs4 import BeautifulSoup
from bs4.element import PageElement

from selenium.webdriver.chrome.webdriver import WebDriver

from itertools import chain


ROOT_URL = "https://www.jobkorea.co.kr/"
ARCHIVE_NAME = make_file_path("saramin.txt")
id_generator_chain = []

In [9]:
def extract_href(el : PageElement):
    path = el.find(name="a")["href"]
    return path.split("/")[2]

def execute_plan_unit(driver : WebDriver, page : int):
    url_page = f"https://www.jobkorea.co.kr/Salary/Index?coKeyword=&tabindex=2&indsCtgrCode=&indsCode=&jobTypeCode=10031&haveAGI=0&orderCode=2&coPage={page}"
    driver.get(url_page)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    el_ul = soup.find(name="ul", attrs={"id": "listCompany"})
    if not el_ul: return
    
    el_li_list = el_ul.find_all(name="li")
    
    id_gen_unit = map(extract_href, el_li_list)
    id_generator_chain.append(id_gen_unit)


In [10]:
def execute_plan(driver : WebDriver):
    for i in range(1, 291):
        execute_plan_unit(driver, i)

In [12]:
def add_lineswap(i):
    return f"{i}\n"

In [11]:
scrap(execute_plan)

In [13]:
# 각 페이지를 실제 파일에 적재
id_generator = chain(*id_generator_chain)
id_generator = map(add_lineswap, id_generator)

saveAsFileWithGenerator("saramin.txt", id_generator)